From modin main and snowpark python repo: `pip install -e .`

In [1]:
import snowflake.snowpark.modin.plugin
import modin.pandas as pd
import numpy as np
import datetime
import pandas as native_pd
from snowflake.snowpark.session import Session; session = Session.builder.create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://snowbiz.okta.com/app/snowflake/exk8wfsfryJIn4IWZ2p7/sso/saml?SAMLRequest=jVLRctowEPwVj%2FqMJbvQEA2QcQMM7tBAsWmmeRO2IBrbktHJMe7XVzaQSR6S6ZvmtHu7d3uju1OROy9cg1ByjDyXIIfLRKVCHsZoG897Q%2BSAYTJluZJ8jBoO6G4yAlbkJQ0q8yw3%2FFhxMI5tJIG2H2NUaUkVAwFUsoIDNQmNgp9L6ruEMgCujZVDF0oKwmo9G1NSjOu6duuvrtIH7BNCMLnFFtVCvqA3EuXnGqVWRiUqv1JOdqYPJDxM%2Bq2ERViF9YX4XcjzCj5T2Z1BQBdxvO6tV1GMnOA63b2SUBVcR1y%2FiIRvN8uzAbAOoofV42K1jWYuSFXvc5bxRBVlZWw3177wnqc4VwdhdxROx6jMRHoaNMntfJc1u2q15HpB%2FvyKi%2BNxOTuAt6rFRmdROJtXwJsgQc7va6J%2Bm2gIUPFQtjkaWyL%2BoEf6PeLHvkf7A%2BoN3f63myfkTG2OQjLTMa9mW4s78ddVmWGdOVaW%2BNU35qdsWO9hr5sfoeyHj09%2BeYMBFG5jQudLoZ0BPfnf%2BUf4LetybA92%2F%2BF0rXKRNM5c6YKZj%2BPxXK%2BriLS376CUF0zkQZpqDmBjynNV32vOjL1poyuO8OSs%2Bv6qJ%2F8A&RelayState=ver%3A1-

# Example 1: Start with small dataset perform some processing, then join with large data

In [2]:
# emphemeral dataframe/lookup table 
# List of U.S. federal holidays
us_holidays = [
    ("New Year's Day", "2025-01-01"),
    ("Martin Luther King Jr. Day", "2025-01-20"),
    ("Presidents' Day", "2025-02-17"),
    ("Memorial Day", "2025-05-26"),
    ("Juneteenth National Independence Day", "2025-06-19"),
    ("Independence Day", "2025-07-04"),
    ("Labor Day", "2025-09-01"),
    ("Columbus Day", "2025-10-13"),
    ("Veterans Day", "2025-11-11"),
    ("Thanksgiving Day", "2025-11-27"),
    ("Christmas Day", "2025-12-25")
]

# Create DataFrame
df_us_holidays = pd.DataFrame(us_holidays, columns=["Holiday", "Date"])

# Convert Date column to datetime
df_us_holidays["Date"] = pd.to_datetime(df_us_holidays["Date"])

In [3]:
assert df_us_holidays.get_backend() == 'Pandas'  # with auto, we should expect this to be local

In [4]:
# Add new columns for transformations
df_us_holidays["Day_of_Week"] = df_us_holidays["Date"].dt.day_name()
df_us_holidays["Month"] = df_us_holidays["Date"].dt.month_name()

In [5]:
df_us_holidays

,Holiday,Date,Day_of_Week,Month
0,New Year's Day,2025-01-01,Wednesday,January
1,Martin Luther King Jr. Day,2025-01-20,Monday,January
2,Presidents' Day,2025-02-17,Monday,February
3,Memorial Day,2025-05-26,Monday,May
4,Juneteenth National Independence Day,2025-06-19,Thursday,June
5,Independence Day,2025-07-04,Friday,July
6,Labor Day,2025-09-01,Monday,September
7,Columbus Day,2025-10-13,Monday,October
8,Veterans Day,2025-11-11,Tuesday,November
9,Thanksgiving Day,2025-11-27,Thursday,November


In [6]:
%%time
#Note that without auto-switching, this took 2.5 min
for index, row in df_us_holidays.iterrows():
    print(f"{row['Holiday']} falls on {row['Day_of_Week']}, {row['Month']} {row['Date'].day}, {row['Date'].year}.")

New Year's Day falls on Wednesday, January 1, 2025.
Martin Luther King Jr. Day falls on Monday, January 20, 2025.
Presidents' Day falls on Monday, February 17, 2025.
Memorial Day falls on Monday, May 26, 2025.
Juneteenth National Independence Day falls on Thursday, June 19, 2025.
Independence Day falls on Friday, July 4, 2025.
Labor Day falls on Monday, September 1, 2025.
Columbus Day falls on Monday, October 13, 2025.
Veterans Day falls on Tuesday, November 11, 2025.
Thanksgiving Day falls on Thursday, November 27, 2025.
Christmas Day falls on Thursday, December 25, 2025.
CPU times: user 111 ms, sys: 5.92 ms, total: 117 ms
Wall time: 136 ms


In [7]:
pandas_df = df_us_holidays.move_to("pandas") # remove this once we have auto-switching

In [8]:
%%time
for index, row in pandas_df.iterrows():
    print(f"{row['Holiday']} falls on {row['Day_of_Week']}, {row['Month']} {row['Date'].day}, {row['Date'].year}.")

New Year's Day falls on Wednesday, January 1, 2025.
Martin Luther King Jr. Day falls on Monday, January 20, 2025.
Presidents' Day falls on Monday, February 17, 2025.
Memorial Day falls on Monday, May 26, 2025.
Juneteenth National Independence Day falls on Thursday, June 19, 2025.
Independence Day falls on Friday, July 4, 2025.
Labor Day falls on Monday, September 1, 2025.
Columbus Day falls on Monday, October 13, 2025.
Veterans Day falls on Tuesday, November 11, 2025.
Thanksgiving Day falls on Thursday, November 27, 2025.
Christmas Day falls on Thursday, December 25, 2025.
CPU times: user 122 ms, sys: 5.43 ms, total: 127 ms
Wall time: 124 ms


### Another mini example: generate synthetic data

In [9]:
%%time
# Generate 10 million transactions (This is also very slow with Snowpark pandas)
import uuid
import pandas
dates = pandas.date_range(start="2025-01-01", end="2025-12-31", freq="D")
num_transactions = 10000000
data = {
    "Transaction_ID": [str(uuid.uuid4()) for _ in range(num_transactions)],
    "Date": np.random.choice(dates, num_transactions),
    "Revenue": np.random.uniform(10, 1000, num_transactions)
}
df_transactions = pd.DataFrame(data)

CPU times: user 18 s, sys: 8.75 s, total: 26.7 s
Wall time: 26.8 s


In [10]:
assert df_transactions.get_backend() == "Pandas"

### 💡 Automatic switching speeds up loops/iterations on small data + inline creation of dataframes

## Example 2: Demonstrate that when data is prefiltered via SQL, the engine choice changes

In [2]:
# Run the following to generate a synthetic dataset with 10M rows of transactions (from 2024-2025 current date)
session.sql('''
CREATE OR REPLACE TABLE revenue_transactions (
    Transaction_ID STRING,
    Date DATE,
    Revenue FLOAT
);''').collect()
session.sql('''SET num_days = (SELECT DATEDIFF(DAY, '2024-01-01', CURRENT_DATE));''').collect()
session.sql('''INSERT INTO revenue_transactions (Transaction_ID, Date, Revenue)
SELECT
    UUID_STRING() AS Transaction_ID,
    DATEADD(DAY, UNIFORM(0, $num_days, RANDOM()), '2024-01-01') AS Date,
    UNIFORM(10, 1000, RANDOM()) AS Revenue
FROM TABLE(GENERATOR(ROWCOUNT => 10000000));
''').collect()

[Row(number of rows inserted=10000000)]

In [3]:
df_transactions = pd.read_snowflake("REVENUE_TRANSACTIONS")

In [4]:
len(df_transactions)

10000000

In [5]:
# df_transactions["DATE"] = pd.to_datetime(df_transactions["DATE"])

In [6]:
df_transactions.groupby("DATE").sum()["REVENUE"]

Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

DATE
2024-01-01    11091771.0
2024-01-02    10997233.0
2024-01-03    11016805.0
2024-01-04    11233712.0
2024-01-05    11167427.0
                 ...    
2025-03-29    11063159.0
2025-03-30    10959584.0
2025-03-31    11088967.0
2025-04-01    10991139.0
2025-04-02    10928739.0
Name: REVENUE, Length: 458, dtype: float64

In [7]:
assert df_transactions.get_backend() == "Snowflake"

In [8]:
# Filter to records in last 7 days
df_transactions_filter = pd.read_snowflake("SELECT * FROM revenue_transactions WHERE Date >= DATEADD( 'days', -7, current_date ) and Date < current_date")

Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
len(df_transactions_filter)

152431

In [10]:
assert df_transactions_filter.get_backend() == "Pandas" # This should work once auto switching is on

In [11]:
df_transactions_filter.groupby("DATE").sum()["REVENUE"]

DATE
2025-03-26    11087078.0
2025-03-27    10980480.0
2025-03-28    10830212.0
2025-03-29    11063159.0
2025-03-30    10959584.0
2025-03-31    11088967.0
2025-04-01    10991139.0
Name: REVENUE, dtype: float64

### 💡 Automatic switching means that pandas work well for both small and large data

# Example 3: 

Forecast using last year's transaction data via a custom apply function

In [28]:
start_date = pd.Timestamp("2025-10-01")
end_date = pd.Timestamp("2025-10-31")

df_transactions_filtered = df_transactions[
    (df_transactions["DATE"] >= start_date - pd.Timedelta(days=365)) &
    (df_transactions["DATE"] < end_date - pd.Timedelta(days=365))
]

In [29]:
len(df_transactions_filtered)

654618

In [32]:
df_transactions_filtered_pandas = df_transactions.move_to("pandas")

In [30]:
# Forecasting function using df.apply

def forecast_revenue(df, start_date, end_date):
    # Filter data from last year
    df_filtered = df[(df["DATE"] >= start_date - pd.Timedelta(days=365)) & (df["DATE"] < start_date)]
    
    # Append future dates to daily_avg for prediction
    future_dates = pd.date_range(start=start_date, end=end_date, freq="D")
    df_future = pd.DataFrame({"DATE": future_dates})

    # Group by DATE and calculate the mean revenue
    daily_avg = df_filtered.groupby("DATE")["REVENUE"].mean().reset_index()
    
    # Merge future dates with predicted revenue, filling missing values
    df_forecast = df_future.merge(daily_avg, on="DATE", how="left")

    import numpy as np
    # Fill missing predicted revenue with overall mean from last year
    df_forecast["PREDICTED_REVENUE"] = np.nan
    df_forecast["PREDICTED_REVENUE"].fillna(daily_avg["REVENUE"].mean(), inplace=True)
    df_forecast["PREDICTED_REVENUE"] = df_forecast["PREDICTED_REVENUE"].astype("float")
    return df_forecast

In [31]:
# Example usage
df_forecast = forecast_revenue(df_transactions, start_date, end_date)
df_forecast

The current operation leads to materialization and can be slow if the data is large!


AttributeError: 'NativeQueryCompiler' object has no attribute 'snowpark_pandas_api_calls'

In [ ]:
# df_forecast["DATE"] = df_forecast["DATE"].dt.strftime('%Y-%m-%d')

In [33]:
def adjust_for_holiday_weekend(row):
    # For national holidays, revenue down 20% since stores are closed. For weekends, revenue is up 20% due to increased activity.
    if row["DATE"].strftime('%Y-%m-%d') in list(df_us_holidays["Date"].dt.strftime('%Y-%m-%d')): 
        return row["PREDICTED_REVENUE"] * 0.8
    elif row["DATE"].weekday() == 5 or row["DATE"].weekday() == 6: #Saturday/Sundays
        return row["PREDICTED_REVENUE"] * 1.2
    return row["PREDICTED_REVENUE"]

In [34]:
df_forecast_pandas = df_forecast.move_to("pandas")

NameError: name 'df_forecast' is not defined

In [ ]:
# Adjust for holidays
df_forecast_pandas["PREDICTED_REVENUE"] = df_forecast_pandas.apply(adjust_for_holiday_weekend, axis=1)

In [ ]:
df_forecast["PREDICTED_REVENUE"] = df_forecast.apply(adjust_for_holiday_weekend, axis=1)

In [ ]:
type(df_forecast)

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()
df_forecast = df_forecast.move_to("pandas")
chart_predicted = alt.Chart(df_forecast).mark_line(color='blue').encode(
    x='monthdate(DATE):T',
    y='PREDICTED_REVENUE:Q',
    tooltip=['DATE', 'PREDICTED_REVENUE']
).properties(title="Predicted Revenue")
chart_predicted

In [ ]:
df_transactions_filtered = df_transactions[
    (df_transactions["DATE"] >= start_date - pd.Timedelta(days=365)) &
    (df_transactions["DATE"] < end_date - pd.Timedelta(days=365))
]
df_transactions_filtered_groupby = df_transactions_filtered.groupby("DATE")["REVENUE"].mean()
# df_transactions_filtered_groupby = df_transactions_filtered_groupby.move_to("pandas")
chart_last_year = alt.Chart(df_transactions_filtered_groupby).mark_line(color='red').encode(
    x='monthdate(DATE):T',
    y='REVENUE:Q',
    tooltip=['DATE', 'REVENUE']
).properties(title="Last Year Revenue")

# Overlay the charts
final_chart = chart_predicted + chart_last_year
final_chart